# Sequence classification

In this exercise, you will get familiar with how to build RNNs in Keras. You will build a recurrent model to classify moview reviews as either positive or negative.

In [26]:
%matplotlib inline

import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb


## IMDB Sentiment Dataset

The large movie review dataset is a collection of 25k positive and 25k negative movie reviews from [IMDB](http://www.imdb.com). Here are some excerpts from the dataset, both easy and hard, to get a sense of why this dataset is challenging:

> Ah, I loved this movie.

> Quite honestly, The Omega Code is the worst movie I have seen in a very long time.

> The wit and pace and three show stopping Busby Berkley numbers put this ahead of the over-rated 42nd Street. 

> There simply was no suspense, precious little excitement and too many dull spots, most of them trying to show why "Nellie" (Monroe) was so messed up.

The dataset can be found at http://ai.stanford.edu/~amaas/data/sentiment/. Since this is a common dataset for RNNs, Keras has a preprocessed version built-in.

In [5]:
# We will limit to the most frequent 20k words defined by max_features, our vocabulary size
max_features = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

The data is preprocessed by replacing words with indexes - review [Keras's docs](http://keras.io/datasets/#imdb-movie-reviews-sentiment-classification). Here's the first review in the training set.

In [6]:
review = X_train[0]
review

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

We can convince ourselves that these are movies reviews, using the vocabulary provided by keras:

In [7]:
word_index = imdb.get_word_index()

First we create a dictionary from index to word, notice that words are indexed starting from the number 3, while the first three entries are for special characters:

In [8]:
index_word = {i+3: w for w, i in word_index.items()}
index_word[0]=''
index_word[1]='start_char'
index_word[2]='oov'

Then we can covert the first review to text:

In [9]:
' '.join([index_word[i] for i in review])

"start_char this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert oov is an amazing actor and now the same being director oov father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the oov of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be pra

#### Exercise 1 - prepare the data

The reviews are different lengths but we need to fit them into a matrix to feed to Keras. We will do this by picking a maximum word length and cutting off words from the examples that are over that limit and padding the examples with 0 if they are under the limit.

Refer to the [Keras docs](http://keras.io/preprocessing/sequence/#pad_sequences) for the `pad_sequences` function. Use `pad_sequences` to prepare both `X_train` and `X_test` to be `maxlen` long at the most.

In [10]:
maxlen = 80
# Pad and clip the example sequences

X_train_pad = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test, maxlen=maxlen)

print('X_train shape:', X_train_pad.shape)
print('X_test shape:', X_test_pad.shape)

X_train shape: (25000, 80)
X_test shape: (25000, 80)


#### Exercise 2 - build an RNN for classifying reviews as positive or negative

Build a single-layer RNN model and train it. You will need to include these parts:

* An `Embedding` layer for efficiently one-hot encoding the inputs - [docs](http://keras.io/layers/embeddings/)
* A recurrent layer. Keras has a [few variants](http://keras.io/layers/recurrent/) you could use. LSTM layers are by far the most popular for RNNs.
* A `Dense` layer for the hidden to output connection.
* A softmax to produce the final prediction.

You will need to decide how large your hidden state will be. You may also consider using some dropout on your recurrent or embedding layers - refers to docs for how to do this.

Training for longer will be much better overall, but since RNNs are expensive to train, you can use 1 epoch to test. You should be able to get > 70% accuracy with 1 epoch. How high can you get?

In [11]:
from keras.layers import Input, Dense, CuDNNLSTM, Embedding
from keras.models import Model

In [12]:
inputs = Input(shape=(80,))
x = Embedding(input_dim=max_features, output_dim=20, input_length=80)(inputs)
x = CuDNNLSTM(32)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 80, 20)            400000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 32)                6912      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 406,978
Trainable params: 406,978
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Design an recurrent model
model.compile('rmsprop', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_pad, y_train, batch_size=32, epochs=1, validation_data=(X_test_pad, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 18s 737us/step - loss: 0.4376 - acc: 0.7894 - val_loss: 0.3672 - val_acc: 0.8363


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=1, validation_data=(X_test, y_test))
loss, acc = model.evaluate(X_test, y_test, batch_size=32)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [16]:
import pandas as pd
pd.Series(y_test[:,1]).value_counts()
# shows we do indeed have 50/50 split, so baseline is 50%.

1.0    12500
0.0    12500
dtype: int64

In [ ]:
model.predict

In [30]:
#my_review = "the movie was horrible i really didn't like it because the actors are not good and the scenery and the photography are quite dull"
my_review = "the movie was great"

In [31]:
my_review_int = [1] + [word_index[w]+3 for w in my_review.split()]
my_review_int

[1, 4, 20, 16, 87]

In [33]:
my_review_pad = sequence.pad_sequences([my_review_int], maxlen=maxlen)

In [34]:
model.predict(my_review_pad)

array([[0.31268847, 0.68731153]], dtype=float32)